<a href="https://colab.research.google.com/github/coldbilberry/repo-gui/blob/main/%D0%A3%D1%80%D0%BE%D0%BA_3_Dataset%2C_Dataloader%2C_BatchNorm%2C_Dropout%2C_%D0%9E%D0%BF%D1%82%D0%B8%D0%BC%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import datetime as dt
from scipy import stats

In [68]:
from sklearn.datasets import fetch_california_housing

california_housing = fetch_california_housing(as_frame = True)

In [69]:
rng = np.random.RandomState(0)
indices = rng.choice(np.arange(california_housing.frame.shape[0]), size = 500,
                     replace = False)
columns_drop = ['Longitude', 'Latitude']
df = california_housing.frame.iloc[indices].drop(columns = columns_drop)
train_y = df['MedHouseVal'].values
df = df.drop(['MedHouseVal'], axis = 1)

In [70]:
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup
14740,4.1518,22.0,5.663073,1.075472,1551.0,4.180593
10101,5.7796,32.0,6.107226,0.927739,1296.0,3.020979
20566,4.3487,29.0,5.930712,1.026217,1554.0,2.910112
2670,2.4511,37.0,4.992958,1.316901,390.0,2.746479
15709,5.0049,25.0,4.319261,1.039578,649.0,1.712401


In [71]:
df.info

<bound method DataFrame.info of        MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup
14740  4.1518      22.0  5.663073   1.075472      1551.0  4.180593
10101  5.7796      32.0  6.107226   0.927739      1296.0  3.020979
20566  4.3487      29.0  5.930712   1.026217      1554.0  2.910112
2670   2.4511      37.0  4.992958   1.316901       390.0  2.746479
15709  5.0049      25.0  4.319261   1.039578       649.0  1.712401
...       ...       ...       ...        ...         ...       ...
11807  3.0321      21.0  4.731083   1.013970      1847.0  2.150175
2594   2.5667      12.0  4.764840   0.968037      1603.0  3.659817
9609   1.7600      29.0  3.664336   1.062937      1697.0  3.955711
13527  1.8750      30.0  5.668950   1.308219      1172.0  2.675799
3263   2.2188      20.0  5.739796   1.178571       521.0  2.658163

[500 rows x 6 columns]>

In [50]:
X_train, X_test, y_train, y_test = train_test_split(df, train_y, test_size = 0.25, random_state = 13)

In [64]:
class MyDataset(torch.utils.data.Dataset):

  def __init__(self, X, y):
    self.X = torch.Tensor(X)
    self.y = torch.from_numpy(y).float()

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, index):
    return(self.X[index], self.y[index])

class Perceptron(nn.Module):

  def __init__(self, input_dim, output_dim, activation = "relu"):
    super(Perceptron, self).__init__()
    self.fc = nn.Linear(input_dim, output_dim)
    self.activation = activation

  def forward(self, x):
    x = self.fc(x)
    if self.activation == "relu":
      return torch.relu(x)
    if self.activation == "sigmoid":
      return torch.sigmoid(x)
    raise RuntimeError


In [72]:
class FeedForward(nn.Module):
  def __init__(self, input_dim, hidden_dim):
    super(FeedForward, self).__init__()
    self.bn1 = nn.BatchNorm1d(input_dim)
    self.fc1 = Perceptron(input_dim, hidden_dim)
    self.bn2 = nn.BathNorm1d(hidden_dim)
    self.dp = nn.Dropout(0.25)
    self.fc2 = Perceptron(hidden_dim, 1, "sigmoid")

  def forward(self, x):
    x = self.bn1(x)
    x = self.fc1(x)
    x = self.dp(x)
    x = self.fc2(x)
    return x.view(-1)

In [ ]:
net = FeedForward(len(X_train[0]), 50)

In [42]:
from os import sched_getscheduler
class Trainer():

  def __init__(self, dataset, net, loss_f, learning_rate=1e-3,
               epoch_amount=10, batch_size=12,
               max_batches_per_epoch=None,
               device='cpu', early_stopping=10,
               optim=torch.optim.Adam,
               sheduler=None, permutate=True):

    self.loss_f = loss_f
    self.learning_rate = learning_rate
    self.epoch_amount = epoch_amount
    self.batch_size = batch_size
    self.max_batches_per_epoch = max.max_batches_per_epoch
    self.device = device
    self.early_stopping = early_stopping
    self.optim = optim
    self.scheduler = scheduler
    self.permutate = permutate
    self.dataset = dataset
    self.start_model = net
    self.best_model = net

    self.train_loss = []
    self.val_loss = []

  def predict(self, X):
    return self.best_model(X)

  def fit(self, X_train, X_test, y_train, y_test):

    Net = self.start_model

    device = torch.device(self.device)

    Net.to(self.device)

    optimizer = self.optim(Net.parameters(), lr=self.learning_rate)

    if self.shceduler is not None:
      scheduler = self.scheduler(optimizer)

    train = self.dataset(X_train, y_train)
    val = self.dataset(X_test, y_test)

    train = DataLoader(train, batch_size=self.batch_size, shuffle=self.permutate)
    val = DataLoader(val, batch_size=self.batch_size, shuffle=False)

    best_val_loss = float('inf')
    best_ep = 0

    for epoch in range(self.epoch_amount):
      start = dt.datetime.now()
      print(f'Эпоха: {epoch}', end='')
      Net.train()
      mean_loss = 0
      batch_n = 0

      for batch_X, target in train:
        if self.max_batches_per_epoch is not None:
          if batch_n >= self.max_batches_per_epoch:
            break
        optimizer.zero_grad()

        batch_X = batch_X.to(self.device)
        target = target.to(self.device)

        predicted_values = Net(batch_X)
        loss = self.loss_f(predicted_values, target)
        loss.backward()
        optimizer.step()

        mean_loss += float(loss)
        batch_n += 1

      mean_loss /= batch_n
      self.train_loss.append(mean_loss)
      print(f'Loss_train: {mean_loss}, {dt.datetime.now() - start} cек')

      Net.eval()
      mean_loss = 0
      batch_n = 0

      with torch.no_grad():
        for batch_X, target in val:
          if self.max_batches_per_epoch is not None:
            if batch_n >= self.max_batches_per_epoch:
              break
        batch_X = batch_X.to(self.device)
        target = target.to(self.device)

        predicted_values = Net(batch_X)
        loss = self.loss_f(predicted_values, target)

        mean_loss += float(loss)
        batch_n += 1

      mean_loss /= batch_n
      self.val_loss.append(mean_loss)
      print(f'Loss_val: {mean_loss}')

      if mean_loss < best_val_loss:
        self.best_model = Net
        best_val_loss = mean_loss
        best_ep = epoch
      elif epoch - best_ep > self.early_stopping:
        print(f'{self.early_stopping} без улучшений. Прекращаем обучение...')
        break
      if self.scheduler is not None:
        scheduler.step()
      print()

In [43]:
class RMSELoss(nn.Module):
  def __init__(self):
    super().__init__()
    self.mse = nn.MSELoss()

  def forward(self, yhat, y):
    return torch.sqrt(self.mse(yhat, y))

In [ ]:
params = {
    'dataset': MyDataset,
    'net': net,
    'batch_size': 1000,
    'epoch_amount': 100,
    'learning_rate': 1e-2,
    'early_stopping': 10,
    'loss_f': RMSELoss(),
    'optim': torch.optim.SGD,
}

In [ ]:
clf = Trainer(**params)
clf.fit(X_train, X_test, y_train, y_test)

In [ ]:
from torch.optim.adam import Adam
net1 = FeedForward(len(X_train[0]), 50)

params = {
    'dataset': MyDataset,
    'net': net1,
    'batch_size': 1000,
    'epoch_amount': 100,
    'learning_rate': 1e-2,
    'early_stopping': 10,
    'loss_f': RMSELoss(),
    'optim': torch.optim.Adam,
}

clf1 = Trainer(**params)
clf1.fit(X_train, X_test, y_train, y_test)

In [ ]:
net2 = FeedForward(len(X_train[0]), 50)

params = {
    'dataset': MyDataset,
    'net': net1,
    'batch_size': 1000,
    'epoch_amount': 1000,
    'learning_rate': 1e-2,
    'early_stopping': 10,
    'loss_f': RMSELoss(),
    'optim':torch.optim.RMSprop,
}

clf2 = Trainer(**params)
clf2.fit(X_train, X_test, y_train, y_test)

Операторы показывают примерно равную эффективность